# Pretraining Models

TODO

## Pretraining the Graph-Based Transformer

TODO

In [1]:
import pandas as pd

N_PROC = 12
MAX_MOLS = 100
GPUS=[0]
EPOCHS=3

In [2]:
smiles = pd.read_table('jupyter/data/chembl_30_ALL.smi', sep='\t', header=0, usecols=('Smiles',)).squeeze('columns').sample(MAX_MOLS)
smiles.shape

(100,)

In [3]:
from utils import initLogger

initLogger('pretraining_graph.log')

In [4]:
from drugex.data.processing import Standardization

standardizer = Standardization(n_proc=N_PROC)
smiles = standardizer.apply(smiles)
len(smiles)

Standardizing molecules (batch processing):   0%|          | 0/1 [00:00<?, ?it/s]

100

In [5]:
from drugex.data.datasets import GraphFragDataSet
import os

graph_input_folder = "data/sets/graph/pretraining"
if not os.path.exists(graph_input_folder):
    os.makedirs(graph_input_folder)

train = GraphFragDataSet(f"{graph_input_folder}/chembl30_train.tsv", rewrite=False)
test = GraphFragDataSet(f"{graph_input_folder}/chembl30_test.tsv", rewrite=False)

In [6]:
from drugex.data.fragments import FragmentCorpusEncoder
from drugex.data.fragments import GraphFragmentEncoder, FragmentPairsSplitter
from drugex.molecules.converters.fragmenters import Fragmenter
from drugex.data.corpus.vocabulary import VocGraph

vocabulary = VocGraph(n_frags=4)
encoder = FragmentCorpusEncoder(
    fragmenter=Fragmenter(4, 4, 'brics'),
    encoder=GraphFragmentEncoder(
        vocabulary
    ),
    pairs_splitter=FragmentPairsSplitter(0.1, 1000, make_unique=False),
    n_proc=N_PROC,
    chunk_size=500
)

encoder.apply(smiles, encodingCollectors=[test, train])

Creating fragment-molecule pairs (batch processing):   0%|          | 0/1 [00:00<?, ?it/s]

Encoding fragment-molecule pairs. (batch processing):   0%|          | 0/1 [00:00<?, ?it/s]

Encoding fragment-molecule pairs. (batch processing):   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
from drugex.training.models.transform.gpt2graph import GraphModel
from drugex.data.corpus.vocabulary import VocGraph
from drugex.training.monitors import FileMonitor

vocabulary = VocGraph(n_frags=4)
model = GraphModel(voc_trg=vocabulary, use_gpus=GPUS)
print(model.gpus)
monitor = FileMonitor('data/models/pretrained/graph/chembl_pretrained')
model.fit(test.asDataLoader(512), train.asDataLoader(512), monitor=monitor, epochs=EPOCHS)

[0]


Fitting model:   0%|          | 0/3 [00:00<?, ?it/s]

Iterating over training batches:   0%|          | 0/1 [00:00<?, ?it/s]

Iterating over validation batches:   0%|          | 0/4 [00:00<?, ?it/s]

tensor([[ 1,  0,  0,  0,  1],
        [ 5,  0,  0,  0,  1],
        [ 5,  1,  0,  1,  1],
        [ 8,  2,  1,  1,  1],
        [ 5,  3,  2,  1,  1],
        [ 2,  4,  3,  2,  1],
        [ 5,  5,  3,  1,  1],
        [ 5,  6,  5,  1,  1],
        [ 5,  7,  6,  2,  1],
        [ 5,  8,  7,  1,  1],
        [ 5,  9,  8,  2,  1],
        [ 5, 10,  9,  1,  1],
        [ 5, 11,  2,  1,  1],
        [39, 11,  6,  1,  1],
        [39, 11, 10,  2,  1],
        [ 5, 12, 12,  0,  2],
        [ 5, 13, 12,  2,  2],
        [ 5, 14, 13,  1,  2],
        [ 5, 15, 14,  2,  2],
        [12, 16, 15,  1,  2],
        [ 8, 17, 16,  1,  2],
        [ 5, 18, 17,  2,  2],
        [ 5, 19, 15,  1,  2],
        [39, 19, 18,  1,  2],
        [ 5, 20, 12,  1,  2],
        [39, 20, 19,  2,  2],
        [14, 21,  0,  3,  0],
        [13, 22, 18,  1,  0],
        [38, 23,  5,  1,  0],
        [ 4, 24, 23,  1,  0],
        [17, 25,  7,  1,  0],
        [16, 26, 20,  1,  0],
        [17, 27, 10,  1,  0],
        [2

Iterating over training batches:   0%|          | 0/1 [00:00<?, ?it/s]

Iterating over validation batches:   0%|          | 0/4 [00:00<?, ?it/s]

KeyboardInterrupt: 